## Reducing Network Latency & Anomaly Detection with Autoencoders

The goal of this project is twofold:
- Reduce the network traffic in the cloud from the gateway layer.
- Detect anomalous data, indicating a faulty sensor or a potential attack.

We use a subset of data collected from Intel Labs between March and April, 2004 (http://db.csail.mit.edu/labdata/labdata.html) as a proof of concept for applying Deep Learning at the IoT Gateway Layer.

In the best case scenario, we can send predicted batches of time series data that are representative of the actual readings in $n/k$ transmissions, where $n$ is the size of our time series set, and $k$ is the batching size.

In the worst case scenario, incorrectly predicted batch values will update what is currently in the cloud at the time of sensor reading. This scenario will perform as well as trivially passing data from the gateway to the cloud unhindered in $n$ transmissions.

### Data Loading

In [1]:
import gzip
import pandas as pd

In [2]:
with gzip.open('data.txt.gz', 'rb') as data_bytes:
    data = pd.read_csv(data_bytes, header=None, sep=' ', parse_dates=[[0, 1]], squeeze=True)
data.columns = ['DATETIME','EPOCH','SENSOR_ID','TEMPERATURE','HUMIDITY','LIGHT','VOLTAGE']
data = data.set_index('DATETIME')

In [3]:
data.shape

(2313682, 6)

### Data Pre-processing

We will consider sensor data between March 1st and March 10th for this experiment, as it contains the majority of the complete data.

In [4]:
data_samp = data.loc['2004-03-01':'2004-03-10'].copy()
data_samp.shape

(892574, 6)

For the purposes of a proof of concept, we will make this a univariate problem (not including DateTime), focusing on Temperature readings.

In [5]:
data_samp.drop(['HUMIDITY','LIGHT','VOLTAGE','EPOCH'], axis=1, inplace=True)

Dropping any Sensor ID's where the value is NaN.

In [6]:
data_samp.dropna(subset=['SENSOR_ID'], inplace=True)

For the sake of out experiment, let us only consider sensors 1-10.

In [7]:
data_samp = data_samp[(data_samp.SENSOR_ID >= 1) & (data_samp.SENSOR_ID <= 10)]

Reshaping the Sensor ID field to an integer value.

In [8]:
data_samp.SENSOR_ID.unique()

array([ 1.,  2.,  3.,  4.,  5.,  6.,  7.,  8.,  9., 10.])

In [9]:
data_samp.SENSOR_ID = data_samp.SENSOR_ID.astype(int)
data_samp.dtypes

SENSOR_ID        int64
TEMPERATURE    float64
dtype: object

In [10]:
data_samp.head()

,SENSOR_ID,TEMPERATURE
DATETIME,,
2004-03-01 00:01:57.130850,1,18.4498
2004-03-01 00:02:50.458234,1,18.4400
2004-03-01 00:04:26.606602,1,18.4400
2004-03-01 00:05:28.379208,1,18.4498
2004-03-01 00:05:50.456126,1,18.4302


We want to measure the temperature at each sensor for a given timestamp, so we will pivot the table, making the column values sensor temperature readings at a given timestamp.

In [11]:
data_samp = data_samp.pivot(columns='SENSOR_ID', values='TEMPERATURE')

In [12]:
data_samp.head()

SENSOR_ID,1,2,3,4,5,6,7,8,9,10
DATETIME,,,,,,,,,,
2004-03-01 00:00:21.445722,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,18.489,NaN
2004-03-01 00:00:22.429139,NaN,18.8712,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2004-03-01 00:00:25.633782,NaN,NaN,NaN,NaN,NaN,NaN,18.7144,NaN,NaN,NaN
2004-03-01 00:00:52.381230,NaN,18.8614,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2004-03-01 00:00:53.317719,NaN,NaN,NaN,NaN,NaN,NaN,18.7046,NaN,NaN,NaN


There appears to be a lot of missing values for temperature readings in our table, due to micro-second DateTime ID's in our time series set. We will resample the data every 2 minutes, taking the mean of the values collected.

In [13]:
data_samp = data_samp.resample('2min').mean()

In [14]:
print('New resampled set has: {} data points.'.format(len(data_samp)))
data_samp.isna().sum()

New resampled set has: 6754 data points.


SENSOR_ID
1      143
2      629
3      114
4      349
5     6754
6      582
7       17
8      753
9       65
10     163
dtype: int64

Clearly sensor 5 is not reading values between our time frame, so we will drop it. Stack brings the prescribed column (SENSOR_ID) into our index, making it easily dropped. We unstack to bring Sensor ID out of the index.

In [15]:
temp_df = data_samp.stack().drop(5, level='SENSOR_ID')
data_samp = temp_df.unstack()

In [16]:
data_samp

SENSOR_ID,1,2,3,4,6,7,8,9,10
DATETIME,,,,,,,,,
2004-03-01 00:00:00,18.449800,18.864667,18.753600,19.11130,18.6752,18.70705,18.386100,18.484100,18.430200
2004-03-01 00:02:00,18.440000,18.848333,18.756867,19.10640,18.6654,18.69235,18.378750,18.469400,NaN
2004-03-01 00:04:00,18.440000,18.832000,18.734000,19.10640,18.6654,18.68500,18.376300,18.475933,18.400800
2004-03-01 00:06:00,NaN,18.851600,18.753600,19.10640,18.6654,18.67765,18.377933,18.482467,18.410600
2004-03-01 00:08:00,18.435100,18.861400,18.773200,19.10150,18.6556,18.68500,18.371400,18.479200,18.433467
...,...,...,...,...,...,...,...,...,...
2004-03-10 08:58:00,22.835300,23.178300,23.776100,23.93045,24.0946,23.99170,25.296733,26.044800,24.473533
2004-03-10 09:00:00,22.879400,23.134200,23.717300,23.92800,24.1044,23.92800,25.395550,26.113400,24.589500
2004-03-10 09:02:00,22.869600,23.121950,23.676467,23.90840,24.1485,23.95740,25.496000,26.280000,24.692400


There are still some missing values, which we can simply deal with by applying linear interpolation to estimate values making our set continuous. Interpolation uses previous values, so for values appearing at the front of our frame (ie. sensor 1) we must make the process bidirectional.

In [17]:
data_samp = data_samp.interpolate(method='linear', limit_direction='both', axis=0)

In [18]:
data_samp.isna().sum()

SENSOR_ID
1     0
2     0
3     0
4     0
6     0
7     0
8     0
9     0
10    0
dtype: int64

In [19]:
data_samp.describe()

SENSOR_ID,1,2,3,4,6,7,8,9,10
count,6754.000000,6754.000000,6754.000000,6754.000000,6754.000000,6754.000000,6754.000000,6754.000000,6754.000000
mean,22.192462,22.126009,22.240772,22.249970,21.786615,21.844309,21.621812,21.801295,21.549061
std,2.395218,1.944178,2.198261,2.049267,1.874288,1.955498,2.174163,2.258517,1.976967
min,17.195400,17.642933,17.577600,18.038200,17.616800,17.789933,10.487300,17.499200,17.548200
25%,20.581300,20.881425,20.767500,20.988000,20.547612,20.574563,20.090075,20.143362,20.174600
50%,22.041500,22.259142,22.213000,22.046400,21.821000,21.742600,21.715650,21.811200,21.613567
75%,23.869200,23.349800,23.709133,23.437387,23.166867,23.178300,22.908800,23.152983,22.813250
max,28.654867,27.416800,28.243267,27.652000,26.534800,26.420467,26.453950,27.162000,25.819400


Now our data set smoothly tracks Temperature over a 2 minute interval without undefined data points. Let's plot our findings for each sensor in our dataframe.

In [20]:
import matplotlib.pyplot as plt

data_samp.plot(subplots=True, legend=True, figsize=(10,20))

array([<matplotlib.axes._subplots.AxesSubplot object at 0x12d501e10>,
      dtype=object)

### Autoencoder

We have collected our time-series data as a vector $\vec{T}$, where $\vec{T}$ contains all of the time samples for each sensor value. This can be represented as:

$\vec{T}=\lbrace\langle s_1\cdots s_n\rangle|1\leq k\leq n, s_k\in S\rbrace$, where $S$ is the set of sensors transmitting to the gateway.

We choose to use an autoencoder for several reasons:

    1.Responds well to seasonal data
    2.Sending the decoder to the cloud is a one-time operation
    3.Trains quickly

The goal of the autoencoder is to send a _representation_ of the input batch that is smaller in size (less bytes) but can be decoded to reveal the same information. Using our time-series data, we will generate a training set that mimics a batch over a windowed time period.

![AE](./img/ae.png)

Our autoencoder input will be an arbitrarily chosen length, with $n$ dimensions for each input. We will iteratively train the encoder in an attempt to reconstruct the original vectors (as close as possible). Once we have a reasonable reconstruction, we can send the yellow nodes to the cloud. Incoming data will be encoded (seen in red) and sent to the cloud as a smaller representation. Let us begin by choosing a window size:

In [21]:
import numpy as np

In [22]:
window = 3

We will turn this time-series problem into a cross-sectional problem by taking a sliding window of vectors and applying the result to a standard autoencoder. First, let us drop values to make our window compatible with an autoencoder input.

In [23]:
import math
# Effectively dropping the last data_samp[0] mod 3 values.
data_window = data_samp[:math.floor(data_samp.shape[0]/window)*window]
data_window

SENSOR_ID,1,2,3,4,6,7,8,9,10
DATETIME,,,,,,,,,
2004-03-01 00:00:00,18.449800,18.864667,18.753600,19.11130,18.6752,18.707050,18.386100,18.484100,18.430200
2004-03-01 00:02:00,18.440000,18.848333,18.756867,19.10640,18.6654,18.692350,18.378750,18.469400,18.415500
2004-03-01 00:04:00,18.440000,18.832000,18.734000,19.10640,18.6654,18.685000,18.376300,18.475933,18.400800
2004-03-01 00:06:00,18.437550,18.851600,18.753600,19.10640,18.6654,18.677650,18.377933,18.482467,18.410600
2004-03-01 00:08:00,18.435100,18.861400,18.773200,19.10150,18.6556,18.685000,18.371400,18.479200,18.433467
...,...,...,...,...,...,...,...,...,...
2004-03-10 08:56:00,22.864700,23.119500,23.830000,23.96965,24.0946,24.071733,25.263250,25.971300,24.403300
2004-03-10 08:58:00,22.835300,23.178300,23.776100,23.93045,24.0946,23.991700,25.296733,26.044800,24.473533
2004-03-10 09:00:00,22.879400,23.134200,23.717300,23.92800,24.1044,23.928000,25.395550,26.113400,24.589500


The new data window size will guarantee the autoencoder input will be consistent. Now we will define a function to create a sliding window over each timestep:

In [24]:
def build_window(arr, window):
    iterator = iter(arr)
    n_windows = len(arr)-window+1
    for i in range(n_windows):
        yield arr[i:i+window]

In [25]:
data_window_slide = list(build_window(data_samp.values, window))

Now our data is in a sliding window of 3 vectors with 9 sensor dimensions. We would like to concatenate the vectors in each windowed batch:

In [26]:
for i in range(len(data_window_slide)):
    data_window_slide[i] = data_window_slide[i].flatten()

In [27]:
data_window_slide[0:3]

[array([18.4498    , 18.86466667, 18.7536    , 19.1113    , 18.6752    ,
        18.70705   , 18.3861    , 18.4841    , 18.4302    , 18.44      ,
        18.84833333, 18.75686667, 19.1064    , 18.6654    , 18.69235   ,
        18.37875   , 18.4694    , 18.4155    , 18.44      , 18.832     ,
        18.734     , 19.1064    , 18.6654    , 18.685     , 18.3763    ,
        18.47593333, 18.4008    ]),
 array([18.44      , 18.84833333, 18.75686667, 19.1064    , 18.6654    ,
        18.69235   , 18.37875   , 18.4694    , 18.4155    , 18.44      ,
        18.832     , 18.734     , 19.1064    , 18.6654    , 18.685     ,
        18.3763    , 18.47593333, 18.4008    , 18.43755   , 18.8516    ,
        18.7536    , 19.1064    , 18.6654    , 18.67765   , 18.37793333,
        18.48246667, 18.4106    ]),
 array([18.44      , 18.832     , 18.734     , 19.1064    , 18.6654    ,
        18.685     , 18.3763    , 18.47593333, 18.4008    , 18.43755   ,
        18.8516    , 18.7536    , 19.1064    , 18.66

### Data Splitting

In [28]:
from sklearn.model_selection import train_test_split

The advantage of taking the time component out is that we are able to randomize our training representations. Each $n\times k$ value (where $k$ is the batch window size) describes a feature of the graph above, but is still a learnable representation in any order. Due to the nature of the data, we do not need to account for seasonal data.

In order to prevent gradients from vanishing, we need to normalize our data, so we will pack our data to values in the range $\lbrack0,1\rbrack$. To accomplish this we can use the following formula:

$\text{norm($x_i$)}=\frac{x_i-\text{min($x$)}}{\text{max($x$)}-\text{min($x$)}}$.

In [29]:
def normalize(data: list):
    normed = data
    min_x = min([min(x) for x in data])
    max_x = max([max(x) for x in data])
    denom = max_x - min_x
    for i in range(len(data)):
        for j in range(len(data[i])):
            normed[i][j] = (data[i][j]-min_x) / denom
    return normed

In [30]:
data_window_norm = normalize(data_window_slide)

In [31]:
data_window_norm[0:3]

[array([0.43828104, 0.46111661, 0.45500315, 0.47469208, 0.45068776,
        0.45244089, 0.43477479, 0.44016902, 0.43720219, 0.43774162,
        0.46021757, 0.45518295, 0.47442237, 0.45014834, 0.45163175,
        0.43437022, 0.43935988, 0.43639306, 0.43774162, 0.45931853,
        0.4539243 , 0.47442237, 0.45014834, 0.45122719, 0.43423537,
        0.4397195 , 0.43558393]),
 array([0.43774162, 0.46021757, 0.45518295, 0.47442237, 0.45014834,
        0.45163175, 0.43437022, 0.43935988, 0.43639306, 0.43774162,
        0.45931853, 0.4539243 , 0.47442237, 0.45014834, 0.45122719,
        0.43423537, 0.4397195 , 0.43558393, 0.43760676, 0.46039737,
        0.45500315, 0.47442237, 0.45014834, 0.45082262, 0.43432527,
        0.44007912, 0.43612335]),
 array([0.43774162, 0.45931853, 0.4539243 , 0.47442237, 0.45014834,
        0.45122719, 0.43423537, 0.4397195 , 0.43558393, 0.43760676,
        0.46039737, 0.45500315, 0.47442237, 0.45014834, 0.45082262,
        0.43432527, 0.44007912, 0.43612335, 0.43

Splitting the dataset:

In [32]:
def split_data(data, test_size=0.1):
    X_train, X_test = train_test_split(data, test_size=test_size)
    return X_train, X_test

In [33]:
train, test = split_data(data_window_norm)

In [34]:
print('Train size: {}, Test size: {}'.format(len(train), len(test)))

Train size: 6076, Test size: 676


We will now define the model:

In [69]:
from keras.layers import Input, Dense
from keras.models import Model

input_size = len(data_window_slide)
n_features = len(data_window_slide[0])
encoding_dim = window
hidden_size = 100

input_layer = Input(shape=(input_size, n_features))
encoded = Dense(encoding_dim, activation='relu')(input_layer)
output_layer = Dense(input_size, activation='sigmoid')(encoded)

autoencoder = Model(input_layer, output_layer)
autoencoder.compile(optimizer='adam', loss='mse')

In [70]:
autoencoder.summary()

Model: "model_12"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_15 (InputLayer)        (None, 6752, 27)          0         
_________________________________________________________________
dense_38 (Dense)             (None, 6752, 3)           84        
_________________________________________________________________
dense_39 (Dense)             (None, 6752, 6752)        27008     
Total params: 27,092
Trainable params: 27,092
Non-trainable params: 0
_________________________________________________________________


Attempt to fit the autoencoder. Note the target of the autoencoder is the training data as well, so the input is supplied as the target. The testing sets will be used for validation.

In [68]:
history = autoencoder.fit(train, train,
    epochs=10,
    batch_size=10,
    validation_data=(test, test)
)

ValueError: Error when checking model input: the list of Numpy arrays that you are passing to your model is not the size the model expected. Expected to see 1 array(s), but instead got the following list of 6076 arrays: [array([[0.56882136],
       [0.58824058],
       [0.58122809],
       [0.59417423],
       [0.56693338],
       [0.57223771],
       [0.54805358],
       [0.54508676],
       [0.54427762],
       [0....

Plotting the results:

In [ ]:
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('Train vs Validation Loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['Train', 'Validation'], loc='upper right')
plt.show()

## LSTM Encoder-Decoder (Future Work)

Given an array of 9 sensor values with readings at every 2 minute interval, we would like to generate a compressed representation of these values, like the section above.

Let us consider the same vector $\vec{T}$ as our sensors temperature representation, but this time as time-series set.

Using the scale from $x_1\cdots x_{Tx}$ as a batch, we could choose a size similar to our sliding window in the previous concept.

This approach is different from above in that it offers a key capability: *prediction*. Using the LSTM approach, we will be able to predict values, and potentially send predictions to the cloud before readings are made. If an anomaly is detected in our actual readings we can update the cloud data, but in the average case this will outperform a traditional gateway. Of course, there are trade-offs to this approach. The upside is there will be no need to send the decoder to the cloud, since we can send batches of predicted values instead. The downside is the performance, and time to train is high.



![AUTOENCODER](./img/autoencoder.png)